In [1]:
import numpy as np
import pandas as pd
import os
import sys
import pickle
from datetime import date
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import re

sys.path.append(os.path.abspath('..'))

In [4]:
df = pd.read_json('C:/Users/lkyoo/PycharmProjects/pythonProject1/infovis-project/yelp_data/yelp_academic_dataset_business.json', lines=True)

# Only keep restaurants, the most popular category/just over 50k businesses
df = df[df['categories'].fillna('no').str.contains('Restaurant')]

# Drop address and attributes as it would require too much processing to be useful for visualization
df.drop(['address', 'attributes', 'postal_code'], axis=1, inplace=True)


In [3]:
# Remove states that only have less than 5 examples
statecounts = df['state'].value_counts()
statecounts = statecounts[statecounts.values < 5]
low_states = statecounts.index

df = df[~df['state'].isin(low_states)]

In [4]:
df.head(5)

,business_id,name,city,state,latitude,longitude,stars,review_count,is_open,categories,hours
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,Philadelphia,PA,39.955505,-75.155564,4.0,80,1,"Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
5,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,Ashland City,TN,36.269593,-87.058943,2.0,6,1,"Burgers, Fast Food, Sandwiches, Food, Ice Crea...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-22:0', '..."
8,k0hlBqXX-Bt0vf1op7Jr1w,Tsevi's Pub And Grill,Affton,MO,38.565165,-90.321087,3.0,19,0,"Pubs, Restaurants, Italian, Bars, American (Tr...",None
9,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,Nashville,TN,36.208102,-86.768170,1.5,10,1,"Ice Cream & Frozen Yogurt, Fast Food, Burgers,...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-21:0', '..."
11,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,Tampa Bay,FL,27.955269,-82.456320,4.0,10,1,"Vietnamese, Food, Restaurants, Food Trucks","{'Monday': '11:0-14:0', 'Tuesday': '11:0-14:0'..."


In [5]:
df.shape

(52281, 11)

In [6]:
days = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']

def hours_to_columns(row):
    if row['hours'] is None:
        for day in days:
            row[day] = None
        return row
    else:
        for day in days:
            if row['hours'].get(day) is None:
                row[day] = 0
            else:
                hours = row['hours'].get(day).split('-')
                open_time = float(hours[0].split(':')[0]) + float(float(hours[0].split(':')[1])/60)
                close_time = float(hours[1].split(':')[0]) + float(float(hours[1].split(':')[1])/60)
                row[day] = abs(close_time - open_time)
        return row


In [7]:
# Process Business Data
df_processed = df.copy()

# Index of business IDs
business_ids = df_processed['business_id'].unique()

# Split hours into 7 columns denoting number of hours open a day
df_processed = df_processed.apply(hours_to_columns, axis=1)
df_processed.set_index('business_id', inplace=True)

df_processed.head(5)

,name,city,state,latitude,longitude,stars,review_count,is_open,categories,hours,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
business_id,,,,,,,,,,,,,,,,,
MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,Philadelphia,PA,39.955505,-75.155564,4.0,80,1,"Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",13.0,13.0,13.0,13.0,14.0,14.0,14.0
CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,Ashland City,TN,36.269593,-87.058943,2.0,6,1,"Burgers, Fast Food, Sandwiches, Food, Ice Crea...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-22:0', '...",0.0,16.0,16.0,16.0,9.0,13.0,14.0
k0hlBqXX-Bt0vf1op7Jr1w,Tsevi's Pub And Grill,Affton,MO,38.565165,-90.321087,3.0,19,0,"Pubs, Restaurants, Italian, Bars, American (Tr...",None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,Nashville,TN,36.208102,-86.768170,1.5,10,1,"Ice Cream & Frozen Yogurt, Fast Food, Burgers,...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-21:0', '...",0.0,15.0,15.0,10.0,10.0,11.0,15.0
eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,Tampa Bay,FL,27.955269,-82.456320,4.0,10,1,"Vietnamese, Food, Restaurants, Food Trucks","{'Monday': '11:0-14:0', 'Tuesday': '11:0-14:0'...",3.0,3.0,3.0,3.0,3.0,5.0,3.0


In [67]:
# Split Business Categories and remove outlier categories (in terms of appearance from the dataset
categories = df_processed['categories'].astype(str).values

categories = ', '.join(categories)

categories = re.sub(r"(\s*?),", ", ", categories)

categories = re.split(', ', categories)

categories = [x.strip() for x in categories]

categories_unique = [*set(categories)]

categories_count = {}

for category in tqdm(categories_unique, desc='Counting categories'):
    categories_count[category] = categories.count(category)

Counting categories: 100%|██████████| 730/730 [00:03<00:00, 194.87it/s]


In [68]:
# Parse category counts into a DF
df_categories = pd.DataFrame.from_dict(categories_count, orient='index', columns=['count'])

# Sort categories descending
df_categories = df_categories.sort_values(by='count', ascending=False)

df_categories.head(10)

,count
Restaurants,52263
Food,15474
Nightlife,8723
Sandwiches,8366
Bars,8337
American (Traditional),8137
Pizza,7092
Fast Food,6472
Breakfast & Brunch,6237
American (New),6096


In [37]:
df_categories.shape

(732, 1)

In [64]:
df_categories.index

Index(['Restaurants', 'Food', 'Nightlife', 'Sandwiches', 'Bars',
       'American Traditional', 'Pizza', 'Fast Food', 'Breakfast & Brunch',
       'American New',
       ...
       'Lahmacun', 'RV Parks', 'Leather Goods', 'Trailer Repair',
       'Gemstones & Minerals', 'Serbo Croatian', 'Pick Your Own Farms',
       'Homeowner Association', 'Pet Boarding', 'Party Characters'],
      dtype='object', length=730)

In [69]:
file = open('../infovis-project/processed_data/business_category_counts.pkl', 'wb')
pickle.dump(df_categories, file)

In [70]:
df_categories.to_csv('../infovis-project/processed_data/business_category_counts.gzip', compression='gzip')

In [11]:
# Processing Reviews
pd_reviews = pd.read_json('../infovis-project/yelp_data/yelp_academic_dataset_review.json', lines=True, chunksize=10000)

today = date.today()

# Add empty columns to business table
df_processed['oldest_review'] = -sys.maxsize
df_processed['newest_review'] = sys.maxsize
df_processed['total_age'] = 0
df_processed['total_liked'] = 0
df_processed['star_1'] = 0
df_processed['star_2'] = 0
df_processed['star_3'] = 0
df_processed['star_4'] = 0
df_processed['star_5'] = 0
df_processed['review_dates'] = ''

count = 0
for chunk in tqdm(pd_reviews, desc='Processing review data'):
    temp = chunk.drop(['review_id', 'user_id', 'text'],axis=1)

    #Filter out rows that dont need to be there (non restaurants)
    temp = temp[temp['business_id'].isin(business_ids)]

    businesses = temp['business_id'].unique()

    # Age reviews
    temp['date'] = pd.to_datetime(temp['date'])
    temp['age'] = round((pd.to_datetime(today) - temp['date']).dt.days/365, 2)

    # Determine if a review is socially liked/supported by checking useful, funny, and cool columns
    temp['liked'] = temp[['useful', 'funny', 'cool']].any(axis=1) * 1

    # Get oldest review, newest review, age sum, number of liked, number for each star value for each business
    for business in businesses:
        temp_business = temp[temp['business_id'] == business]
        age_max = temp_business['age'].max()
        age_min = temp_business['age'].min()
        total_age = temp_business['age'].sum()
        total_liked = temp_business['liked'].sum()
        stars = temp_business['stars'].value_counts()
        dates = list(temp_business['date'].values)

        #Compare and update min/max
        if age_max > float(df_processed.at[business, 'oldest_review']):
            df_processed.at[business, 'oldest_review'] = age_max

        if age_min < float(df_processed.at[business, 'newest_review']):
            df_processed.at[business, 'newest_review'] = age_min

        #Add to total age and liked
        df_processed.at[business, 'total_age'] = float(df_processed.at[business, 'total_age']) + total_age
        df_processed.at[business, 'total_liked'] = float(df_processed.at[business, 'total_liked']) + total_liked
        df_processed.at[business, 'review_dates'] = list(df_processed.at[business, 'review_dates']) + dates

        #Star columns
        for key in stars.keys():
            df_processed.at[business, 'star_'+str(key)] = float(df_processed.at[business, 'star_'+str(key)]) + stars[key]

        count+=1

        if count == 3:
            break


Processing review data: 700it [46:45,  4.01s/it]


In [12]:
# Derived columns: average review age, hours per week, estimated business age
df_processed['avg_review_age'] = round(df_processed['total_age']/df_processed['review_count'],2)
df_processed['est_business_age'] = round(abs(df_processed['oldest_review'] - df_processed['newest_review']),2)
df_processed['hours_per_week'] = df_processed['Monday'] + df_processed['Tuesday'] + df_processed['Wednesday'] + df_processed['Thursday'] + df_processed['Friday'] + df_processed['Saturday'] + df_processed['Sunday']

df_processed.head(10)

,name,city,state,latitude,longitude,stars,review_count,is_open,categories,hours,...,total_liked,star_1,star_2,star_3,star_4,star_5,review_dates,avg_review_age,est_business_age,hours_per_week
business_id,,,,,,,,,,,,,,,,,,,,,
MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,Philadelphia,PA,39.955505,-75.155564,4.0,80,1,"Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",...,53,5,4,9,31,37,"[2013-10-05T15:19:06.000000000, 2013-10-25T01:...",7.45,13.66,94.0
CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,Ashland City,TN,36.269593,-87.058943,2.0,6,1,"Burgers, Fast Food, Sandwiches, Food, Ice Crea...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-22:0', '...",...,0,3,1,0,2,0,"[2012-12-18T08:45:44.000000000, 2013-07-08T00:...",5.07,8.22,84.0
k0hlBqXX-Bt0vf1op7Jr1w,Tsevi's Pub And Grill,Affton,MO,38.565165,-90.321087,3.0,19,0,"Pubs, Restaurants, Italian, Bars, American (Tr...",None,...,9,6,2,1,7,3,"[2014-02-25T22:34:59.000000000, 2013-01-29T04:...",9.47,2.25,NaN
bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,Nashville,TN,36.208102,-86.768170,1.5,10,1,"Ice Cream & Frozen Yogurt, Fast Food, Burgers,...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-21:0', '...",...,2,7,1,1,0,1,"[2016-11-11T04:01:28.000000000, 2011-07-01T23:...",4.60,9.80,76.0
eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,Tampa Bay,FL,27.955269,-82.456320,4.0,10,1,"Vietnamese, Food, Restaurants, Food Trucks","{'Monday': '11:0-14:0', 'Tuesday': '11:0-14:0'...",...,8,1,0,2,2,6,"[2019-04-04T16:03:00.000000000, 2018-10-23T00:...",3.35,3.45,23.0
il_Ro8jwPlHresjw9EGmBg,Denny's,Indianapolis,IN,39.637133,-86.127217,2.5,28,1,"American (Traditional), Restaurants, Diners, B...","{'Monday': '6:0-22:0', 'Tuesday': '6:0-22:0', ...",...,8,15,2,1,7,4,"[2016-05-08T08:49:25.000000000, 2017-08-30T10:...",4.66,7.11,112.0
0bPLkL0QhhPO5kt1_EXmNQ,Zio's Italian Market,Largo,FL,27.916116,-82.760461,4.5,100,0,"Food, Delis, Italian, Bakeries, Restaurants","{'Monday': '10:0-18:0', 'Tuesday': '10:0-20:0'...",...,61,7,4,1,18,74,"[2018-04-28T00:32:12.000000000, 2015-08-30T18:...",7.19,9.40,58.0
MUTTqe8uqyMdBl186RmNeA,Tuna Bar,Philadelphia,PA,39.953949,-75.143226,4.0,245,1,"Sushi Bars, Restaurants, Japanese","{'Tuesday': '13:30-22:0', 'Wednesday': '13:30-...",...,131,13,12,29,50,142,"[2018-01-22T01:47:25.000000000, 2018-03-24T14:...",3.57,4.14,53.0
ROeacJQwBeh05Rqg7F6TCg,BAP,Philadelphia,PA,39.943223,-75.162568,4.5,205,1,"Korean, Restaurants","{'Monday': '11:30-20:30', 'Tuesday': '11:30-20...",...,97,4,6,28,49,117,"[2016-10-27T01:10:32.000000000, 2014-03-15T19:...",5.93,8.10,54.0


In [13]:
df_checkin = pd.read_json('../infovis-project/yelp_data/yelp_academic_dataset_checkin.json', lines=True)

df_checkin.head(5)

,business_id,date
0,---kPU91CF4Lq2-WlRu9Lw,"2020-03-13 21:10:56, 2020-06-02 22:18:06, 2020..."
1,--0iUa4sNDFiZFrAdIWhZQ,"2010-09-13 21:43:09, 2011-05-04 23:08:15, 2011..."
2,--30_8IhuyMHbSOcNWd6DQ,"2013-06-14 23:29:17, 2014-08-13 23:20:22"
3,--7PUidqRWpRSpXebiyxTg,"2011-02-15 17:12:00, 2011-07-28 02:46:10, 2012..."
4,--7jw19RH9JKXgFohspgQw,"2014-04-21 20:42:11, 2014-04-28 21:04:46, 2014..."


In [14]:
df_checkin.rename(columns={'date':'checkins'}, inplace=True)

df_processed = pd.merge(left=df_processed, right=df_checkin, how='inner', on='business_id')

df_processed.isnull().sum()

business_id        0
name               0
city               0
state              0
latitude           0
longitude          0
stars              0
review_count       0
is_open            0
categories         0
hours           7276
dtype: int64

In [17]:
df_processed.head(5)

,business_id,name,city,state,latitude,longitude,stars,review_count,is_open,categories,...,star_1,star_2,star_3,star_4,star_5,review_dates,avg_review_age,est_business_age,hours_per_week,checkins
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,Philadelphia,PA,39.955505,-75.155564,4.0,80,1,"Restaurants, Food, Bubble Tea, Coffee & Tea, B...",...,5,4,9,31,37,"[2013-10-05T15:19:06.000000000, 2013-10-25T01:...",7.45,13.66,94.0,"2010-08-18 17:05:36, 2010-11-25 17:45:31, 2011..."
1,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,Ashland City,TN,36.269593,-87.058943,2.0,6,1,"Burgers, Fast Food, Sandwiches, Food, Ice Crea...",...,3,1,0,2,0,"[2012-12-18T08:45:44.000000000, 2013-07-08T00:...",5.07,8.22,84.0,"2012-12-16 05:27:33, 2013-02-15 04:00:46, 2013..."
2,k0hlBqXX-Bt0vf1op7Jr1w,Tsevi's Pub And Grill,Affton,MO,38.565165,-90.321087,3.0,19,0,"Pubs, Restaurants, Italian, Bars, American (Tr...",...,6,2,1,7,3,"[2014-02-25T22:34:59.000000000, 2013-01-29T04:...",9.47,2.25,NaN,"2012-04-16 22:28:12, 2012-04-20 22:38:55, 2012..."
3,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,Nashville,TN,36.208102,-86.768170,1.5,10,1,"Ice Cream & Frozen Yogurt, Fast Food, Burgers,...",...,7,1,1,0,1,"[2016-11-11T04:01:28.000000000, 2011-07-01T23:...",4.60,9.80,76.0,"2011-01-17 15:31:21, 2011-02-23 21:28:08, 2011..."
4,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,Tampa Bay,FL,27.955269,-82.456320,4.0,10,1,"Vietnamese, Food, Restaurants, Food Trucks",...,1,0,2,2,6,"[2019-04-04T16:03:00.000000000, 2018-10-23T00:...",3.35,3.45,23.0,"2018-07-19 20:23:33, 2019-07-12 15:37:41, 2019..."


In [18]:
df_processed.to_csv('../infovis-project/processed_data/business_review_fused.gzip', compression='gzip')